<a href="https://colab.research.google.com/github/norgaston/laboratorio3-mlbd/blob/main/Laboratorio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Laboratorio 3 - Clasificador zonas según el peligro**

**ELEMENTOS DE APRENDIZAJE DE MÁQUINA Y BIG DATA**

Carrera:  T.U. TECNOLOGÍAS DE PROGRAMACIÓN SEDE PUNTA ALTA

Facultad: Facultad de la Micro, Pequeña y Mediana Empresa (UPSO)

Docente: Valentín Barco

Cuatrimestre/Año: 2° Cuatrimestre 2023


En el laboratorio, nos centraremos en un conjunto de datos que incluye registros de
crímenes de 1973 en distintos estados de EE. UU., así como el porcentaje de la población
que reside en zonas urbanas en esos estados. El propósito es clasificar los estados en
diferentes categorías basadas en estos factores.

### **Integrantes del Grupo 12: Dolores Ponce y Gaston Ponce**

## **Bibliotecas necesarias**

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go

## **Carga y análisis exploratorio de los datos**

In [2]:
# Cargar el conjunto de datos
ruta = '/content/Crimenes.csv'  # Ruta desde donde voy a cargar el dataset
data = pd.read_csv(ruta, encoding='latin-1')

In [3]:
data.head()

,Estado,Asesinatos,Asaltos,Poblacion Urbana,Violaciones
0,Alabama,13.2,236,58,21.2
1,Alaska,10.0,263,48,44.5
2,Arizona,8.1,294,80,31.0
3,Arkansas,8.8,190,50,19.5
4,California,9.0,276,91,40.6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Estado            50 non-null     object 
 1   Asesinatos        50 non-null     float64
 2   Asaltos           50 non-null     int64  
 3   Poblacion Urbana  50 non-null     int64  
 4   Violaciones       50 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.1+ KB


In [5]:
data.describe()

,Asesinatos,Asaltos,Poblacion Urbana,Violaciones
count,50.00000,50.000000,50.000000,50.000000
mean,7.78800,170.760000,65.540000,21.232000
std,4.35551,83.337661,14.474763,9.366385
min,0.80000,45.000000,32.000000,7.300000
25%,4.07500,109.000000,54.500000,15.075000
50%,7.25000,159.000000,66.000000,20.100000
75%,11.25000,249.000000,77.750000,26.175000
max,17.40000,337.000000,91.000000,46.000000


In [6]:
data.isnull().sum()

Estado              0
Asesinatos          0
Asaltos             0
Poblacion Urbana    0
Violaciones         0
dtype: int64

## **Preparado de los datos**

In [7]:
# Normalizar las características (escalar las características para que tengan una media de cero y una desviación estándar de uno)
scaler = StandardScaler()
data[['Asesinatos', 'Asaltos', 'Violaciones']] = scaler.fit_transform(data[['Asesinatos', 'Asaltos', 'Violaciones']])

## **Búsqueda del número óptimo de clústeres**

In [8]:
# Calcular la inercia para diferentes cantidades de clústeres
inertia = []
for n_clusters in range(1, 10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=20)
    kmeans.fit(data[['Asesinatos', 'Asaltos', 'Violaciones']])
    inertia.append(kmeans.inertia_)

# Crear el gráfico interactivo con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, 10)), y=inertia, mode='lines+markers'))
fig.update_layout(title='Curva del Codo',
                  xaxis_title='Número de Clústeres',
                  yaxis_title='Inercia')
fig.show()


Elijo crear el modelo con 3 clústeres porque se podría proporcionar una segmentación más útil para entender las diferencias en la peligrosidad entre las zonas.

## **Creación del modelo de clasificación**

In [9]:
# Crear un modelo K-Means con 3 clústeres (zonas de baja peligrosidad, media peligrosidad y alta peligrosidad)
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)

# Aplicar K-Means a los datos
data['Cluster'] = kmeans.fit_predict(data[['Asesinatos', 'Asaltos', 'Violaciones']])


## **Gráfico de los resultados**

In [10]:
# Contar la cantidad de estados en cada cluster
cluster_counts = data['Cluster'].value_counts().sort_index()

# Crear un gráfico de barras con Plotly Graph Objects
fig = go.Figure()

# Crear las variables con los colores a utilizar para representar el nivel de peligrosidad en cada estado
red = '#FF9E9E'
yellow = '#FFFFB8'
green = '#9EFFA3'

# Agregar una barra para cada cluster
for cluster, count in cluster_counts.items():
    fig.add_trace(go.Bar(x=[cluster], y=[count], marker_color=red if cluster == 2 else yellow if cluster == 1 else green))

# Configurar el diseño del gráfico
fig.update_layout(title_text='Cantidad de Estados en Cada Cluster',
                  xaxis_title='Peligrosidad',
                  yaxis_title='Cantidad de Estados',
                  xaxis=dict(tickvals=[0, 1, 2], ticktext=['Baja ', 'Moderada', 'Alta']))
# Desactivar la leyenda
fig.update_layout(showlegend=False)
# Mostrar el gráfico
fig.show()


In [11]:
# Asignar colores a los clústeres (0: verde, 1: amarillo, 2: rojo)
data['Color'] = data['Cluster'].map({0: green, 1: yellow, 2: red})

# Ordenar los estados por peligrosidad y luego por población
data = data.sort_values(by=['Cluster', 'Poblacion Urbana'], ascending=[True, True])

# Crear un gráfico de barras agrupadas con Plotly Graph Objects
fig = go.Figure()

# Agregar una barra para cada estado
for _, row in data.iterrows():
    fig.add_trace(go.Bar(
        x=[row['Estado']],
        y=[row['Poblacion Urbana']],
        marker=dict(color=row['Color']),
        showlegend=False,
    ))

# Configurar el diseño del gráfico de barras
fig.update_layout(
    title='Clasificación de Estados por Peligrosidad',
    xaxis_title='Estado',
    yaxis_title='Población Urbana',
)

# Agregar leyenda
legend_items = [
    go.Scatter(x=[None], y=[None], marker=dict(color=green), name='Baja Peligrosidad'),
    go.Scatter(x=[None], y=[None], marker=dict(color=yellow), name='Mediana Peligrosidad'),
    go.Scatter(x=[None], y=[None], marker=dict(color=red), name='Alta Peligrosidad'),
]
for item in legend_items:
    fig.add_trace(item)

fig.show()

# Ordenar los estados por peligrosidad y luego por población
data = data.sort_values(by=['Cluster', 'Poblacion Urbana'], ascending=[True, True])

# Crear un DataFrame para mostrar los resultados
resultados = pd.DataFrame({
    'Estado': data['Estado'],
    'Poblacion Urbana': data['Poblacion Urbana'],
    'Peligrosidad': data['Cluster'].map({0: 'Baja', 1: 'Moderada', 2: 'Alta'}),
})

resultados

,Estado,Poblacion Urbana,Peligrosidad
44,Vermont,32,Baja
47,West Virginia,39,Baja
33,North Dakota,44,Baja
40,South Dakota,45,Baja
18,Maine,51,Baja
11,Idaho,54,Baja
28,New Hampshire,56,Baja
14,Iowa,57,Baja
26,Nebraska,62,Baja
22,Minnesota,66,Baja


## Conclusiones:

El análisis de clusters ha revelado un fascinante panorama sobre la peligrosidad en diferentes estados de EE. UU., considerando variables clave como asesinatos, asaltos y violaciones. Los estados fueron agrupados en tres categorías distintas, cada una con sus propias características.

### Resultados del Análisis de Clusters:

1. **Baja Peligrosidad (Cluster 0):**
   - Incluye estados como Vermont, West Virginia y North Dakota.
   - Se destacan por tasas bajas de crímenes violentos y una población urbana relativamente menor.
   - Aunque suene paradójico, estos estados demuestran que es posible mantener una baja peligrosidad incluso con una población urbana modesta.

2. **Moderada Peligrosidad (Cluster 1):**
   - En este cluster encontramos estados como Mississippi, North Carolina y Alaska.
   - Presentan tasas moderadas de crímenes violentos y una variabilidad interesante en la población urbana.
   - La complejidad de este cluster sugiere la influencia de múltiples factores en la seguridad.

3. **Alta Peligrosidad (Cluster 2):**
   - Incluye estados como Arkansas, Kentucky y Montana.
   - Caracterizados por tasas más altas de crímenes violentos, algunos con una población urbana considerable.
   - Este cluster ofrece una perspectiva valiosa sobre los desafíos específicos que enfrentan los estados con poblaciones urbanas más significativas.

### Consideraciones Adicionales:

- **Distribución de Estados por Cluster:**
  - Baja Peligrosidad (Cluster 0): 14 estados.
  - Moderada Peligrosidad (Cluster 1): 19 estados.
  - Alta Peligrosidad (Cluster 2): 17 estados.

Estos resultados no solo brindan información esencial para la formulación de políticas de seguridad, sino que también invitan a una reflexión más profunda sobre la complejidad de los factores que contribuyen a la seguridad en diferentes entornos.

